### 우리 사이 스벅 - 아이디어 검증
[부동산 - 스타벅스 매장 데이터와 지도](https://financedata.github.io/posts/python_starbucks_map.html)에서 인사이트를 얻었고, 아래는 그 코드를 변경하여 실행해보았습니다.

- [Folium Quick Start](http://folium.readthedocs.io/en/latest/quickstart.html)
- [Folium API Doc](http://python-visualization.github.io/folium/docs-master)

In [40]:
import requests
import json
import pandas as pd
from pandas.io.json import json_normalize
import folium

API_KEY = 'C75A0E7E-CB1F-300E-8032-95C597F3E226'

In [41]:
# 주소를 좌표로 변환
def get_geoloc(addr):
    URI = f'http://apis.vworld.kr/new2coord.do?q={addr}&apiKey={API_KEY}&domain=http://map.vworld.kr/&output=json'
    r = requests.get(URI)
    r_j = r.json()
    # TODO - 예외처리(없는 주소, API key 오류)
    location = (float(r_j['EPSG_4326_Y']), float(r_j['EPSG_4326_X']))
    return location

In [42]:
from collections import namedtuple
Geoloc = namedtuple('Geoloc', ['loc', 'tag'])

### 각자의 출발지 주소를 좌표로 변환

In [43]:
# namedtuple을 이용하여 (주소, 태그)를 (좌표, 태그)로 변경
addr_list = [('서울시 강남구 봉은사로7길 20-1', '루카집'), ('서울시 강남구 테헤란로10길 10', 'Gin'), ('서울시 광진구 천호대로116길 108-11', '빵형')]
geoloc_list = [Geoloc(get_geoloc(addr), tag) for addr, tag in addr_list]
geoloc_list

[Geoloc(loc=(37.507038913, 127.027463426), tag='루카집'),
 Geoloc(loc=(37.498402165, 127.032264518), tag='Gin'),
 Geoloc(loc=(37.553302917, 127.081286591), tag='빵형')]

### 출발지 좌표들의 무게 중심 좌표를 구함

In [44]:
def get_center_of_mass(loc_list):
    count = len(loc_list)
    sum_lat = 0
    sum_lot = 0
    for loc in loc_list:
        sum_lat += loc[0]
        sum_lot += loc[1]
    return (sum_lat/count, sum_lot/count)

In [45]:
center_of_mass = get_center_of_mass([geoloc.loc for geoloc in geoloc_list])
print(center_of_mass)
# mac 계산기로 직접 계산해봄
mac_com = (37.5195813317, 127.047004845)
print(mac_com)
# 유의미한 차이가 없음을 확인함

(37.51958133166667, 127.047004845)
(37.5195813317, 127.047004845)


In [46]:
import random

map_osm = folium.Map(location=center_of_mass, zoom_start=13)
for loc, tag in geoloc_list:
    r = lambda: random.randint(0,255)
    random_color = '#%02X%02X%02X' % (r(),r(),r())
    folium.CircleMarker(loc, popup=tag, radius=300, color=random_color, fill=True, fill_color=random_color).add_to(map_osm)

folium.Marker(center_of_mass, popup='무게 중심', icon=folium.Icon(icon='cloud')).add_to(map_osm)
folium.CircleMarker(center_of_mass, popup='무게 중심', radius=100, color='#ff86cc', fill=True, fill_color='#ff86cc').add_to(map_osm)
map_osm

- 음... 세 원의 교차 지점에 무게 중심이 나올 줄 알았다. 무게중심에 대해서 더 알아봐야겠다.
- 그리고 Circle marker의 radius가 zoom state에 따라 상대적이라서 특정 줌 상태에서만 intersection을 보여주고 있음
- 절대값 옵션이 있는지 확인해보고, 또는 새로 마커를 그리거나 업데이트 하는 방법을 고려해봐야겠다.

### 서울시 스타벅스 정보 가져오기

In [47]:
from folium import plugins

data_seoul = {
    'ins_lat': '33.4996213', # 제주 시청의 좌표
    'ins_lng': '126.5311884',
    'p_sido_cd':'01', # 01=서울시, 08=경기 ... 16=제주
    'p_gugun_cd':'',  # 세부지역 (지정하지 않으면 시/도 전체)
    'in_biz_cd':'',
    'set_date':'',
    'iend':'1000',
}
    
url = 'https://www.istarbucks.co.kr/store/getStore.do'
r = requests.post(url, data=data_seoul)
df_seoul = json_normalize(json.loads(r.text), 'list')
df_seoul = df_seoul[['s_name', 'lat', 'lot', 'sido_name', 'gugun_name', 'doro_address', 'tel']]
df_seoul.head(20)

,s_name,lat,lot,sido_name,gugun_name,doro_address,tel
0,금천시흥,37.448624,126.903183,서울,금천구,서울특별시 금천구 시흥대로 164 (시흥동),02-805-3360
1,청계산입구역,37.447323,127.056250,서울,서초구,서울특별시 서초구 청계산로 203 (신원동),02-577-3253
2,금천독산,37.46914,126.895246,서울,금천구,서울특별시 금천구 두산로 66 (독산동),02-758-8765
3,독산사거리,37.471511,126.898775,서울,금천구,서울특별시 금천구 시흥대로 426 (독산동),02-838-3444
4,하이브랜드,37.46330766,127.0368447,서울,서초구,"서울특별시 서초구 매헌로 16, 리빙관 101호 (양재동)",02-2155-4117
5,가산브이타워,37.477221,126.883683,서울,금천구,서울특별시 금천구 가산디지털1로 128 (가산동),02-758-8371
6,W-Mall,37.47730499,126.8876908,서울,금천구,서울특별시 금천구 디지털로 188 (가산동),02-758-8196
7,마리오아울렛,37.478701,126.885076,서울,금천구,서울특별시 금천구 디지털로9길 23 (가산동),02-758-8542
8,가산그레이트,37.4794491,126.88747680000006,서울,금천구,"서울특별시 금천구 디지털로9길 32, 갑을그레이트밸리 1층 (가산동)",02-758-8346
9,가산디지털,37.4798346,126.88249969999992,서울,금천구,서울특별시 금천구 가산디지털1로 168 (가산동),02-758-8147


### 지도에 스타벅스 지점을 묶어서 표시
- marker cluster를 사용

In [48]:
markerCluster = folium.plugins.MarkerCluster().add_to(map_osm)
for ix, row in df_seoul.iterrows():
    loc = (float(row['lat']), float(row['lot']))
#     print(f'{loc} {row["s_name"]} {type(row["lat"])}')
    folium.Marker(loc, popup=row['s_name']).add_to(markerCluster)

map_osm

### 번외 - 좌표 간의 거리 구하기

In [49]:
from math import sin, cos, sqrt, atan2, radians

# approximate radius of earth in km
R = 6373.0

lat1 = radians(52.2296756)
lon1 = radians(21.0122287)
lat2 = radians(52.406374)
lon2 = radians(16.9251681)

dlon = lon2 - lon1
dlat = lat2 - lat1

a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
c = 2 * atan2(sqrt(a), sqrt(1 - a))

distance = R * c

print("Result:", distance)
print("Should be:", 278.546, "km")

Result: 278.54558935106695
Should be: 278.546 km


### 이전 코드

In [50]:
addr_list = [('서울시 강남구 봉은사로7길 20-1', '루카집'), ('과천시 별양로 85', '호선생님')]
서울_시청=(37.56629, 126.979808)
map_osm = folium.Map(location=서울_시청, zoom_start=13)
for addr, tag in addr_list:
    API_KEY = '767B7ADF-10BA-3D86-AB7E-02816B5B92E9'
    URI = 'http://apis.vworld.kr/new2coord.do?q=%s&apiKey=%s&domain=http://map.vworld.kr/&output=json' % (addr, API_KEY)
    r =requests.get(URI)
    r_j = r.json()

    print(r_j)
    location = (float(r_j['EPSG_4326_Y']), float(r_j['EPSG_4326_X']))
    print("%s %s" % (location, tag))
    folium.CircleMarker(location, popup=tag, radius=200, color='#3186cc',
                    fill_color='#3186cc',).add_to(map_osm)
    
map_osm

from folium import plugins

data = {
    'ins_lat': '33.4996213', # 제주 시청의 좌표
    'ins_lng': '126.5311884',
    'p_sido_cd':'08', # 01=서울시, 08=경기 ... 16=제주
    'p_gugun_cd':'',  # 세부지역 (지정하지 않으면 시/도 전체)
    'in_biz_cd':'',
    'set_date':'',
    'iend':'1000',
}
    
url = 'https://www.istarbucks.co.kr/store/getStore.do'
r = requests.post(url, data=data)
df = json_normalize(json.loads(r.text), 'list')

markerCluster = folium.plugins.MarkerCluster().add_to(map_osm)
for ix, row in df.iterrows():
    location = (float(row['lat']), float(row['lot']))
    print("%s %s %s" % (location, row['s_name'], type(row['lat'])))
    folium.Marker(location, popup=row['s_name']).add_to(markerCluster)

map_osm

{'EPSG_4326_X': '127.027463426', 'EPSG_4326_Y': '37.507038913', 'JUSO': '서울특별시 강남구 봉은사로7길 20-1'}
(37.507038913, 127.027463426) 루카집
{'EPSG_4326_X': '126.994403377', 'EPSG_4326_Y': '37.426134902', 'JUSO': '경기도 과천시 별양로 85 (주공아파트)'}
(37.426134902, 126.994403377) 호선생님
(36.9547073, 127.04916030000004) 평택험프리 <class 'str'>
(36.957324, 127.035458) 평택험프리 트룹몰 <class 'str'>
(36.960753, 127.037315) 평택험프리 메인몰 <class 'str'>
(36.99159, 127.08524) 평택AK플라자 <class 'str'>
(36.9918478, 127.08850410000002) 평택로데오 <class 'str'>
(37.007599, 127.270743) 안성중앙로 <class 'str'>
(37.066397, 127.062939) 평택송탄 <class 'str'>
(37.089517, 127.05814) 오산AB <class 'str'>
(37.1426829, 127.0766272) 오산IC DT <class 'str'>
(37.1486248, 127.0741288) 오산시청 <class 'str'>
(37.197988, 127.098448) 동탄역 <class 'str'>
(37.201584, 127.073193) 동탄 <class 'str'>
(37.204774, 127.071884) 동탄광장 <class 'str'>
(37.2082443, 127.0638898) 동탄센트럴파크 <class 'str'>
(37.21261, 127.038187) 화성병점DT <class 'str'>
(37.20663, 127.111333) 동탄영천 <class 'str'>
(37.2147

(37.66011, 126.76991) 라페스타광장 <class 'str'>
(37.658155, 126.837639) 원당DT <class 'str'>
(37.662238, 126.743981) 킨텍스이마트 <class 'str'>
(37.662091, 126.8009) 일산풍동DT <class 'str'>
(37.663813, 126.891317) 고양삼송 <class 'str'>
(37.669903, 126.762657) 일산주엽 <class 'str'>
(37.671554, 126.758992) 주엽사거리 <class 'str'>
(37.6479682, 127.1233987) 별내 <class 'str'>
(37.650317, 127.112364) 별내불암로 <class 'str'>
(37.673769, 126.786883) 풍산이마트 <class 'str'>
(37.6753761, 126.7481894) 대화역 <class 'str'>
(37.6773867, 126.76756990000001) 일산후곡 <class 'str'>
(37.6779381, 126.812161) 일산식사 <class 'str'>
(37.6477605, 127.23469380000006) 평내DT <class 'str'>
(37.6540359, 127.24287989999993) 평내호평역 <class 'str'>
(37.69192, 126.756991) 일산덕이DT <class 'str'>
(37.695265, 126.762135) 일산탄현 <class 'str'>
(37.699706, 126.75709) 일산가구단지 <class 'str'>
(37.71203, 126.744499) 파주운정 <class 'str'>
(37.7095394, 126.90529470000001) 고양DT <class 'str'>
(37.722373, 127.047477) 의정부회룡역DT <class 'str'>
(37.752207, 126.765613) 파주금릉역 <class 'str'>
(37.

## 스타벅스 API 탐구

### 광역시, 도를 입력하여 군구 지역 코드 가져오기

In [51]:
# 지역 코드 가져오기
data_gugun = {
    'rndCod': 'PZ6IH5S3HM',
    'sido_cd': '08' # 경기
}
url = 'http://www.istarbucks.co.kr/store/getGugunList.do'
r = requests.post(url, data_gugun)
df_gugun = json_normalize(json.loads(r.text), 'list')
df_gugun.dropna(axis=0)
df_gugun = df[['gugun_nm', 'gugun_cd']]
print(len(df_gugun))
df_gugun.head(10)

216


,gugun_nm,gugun_cd
0,None,None
1,None,None
2,None,None
3,None,None
4,None,None
5,None,None
6,None,None
7,None,None
8,None,None
9,None,None


### 전체 목록에서 필터링 한것과 API를 통해 지역 필터링 결과가 같은지 확인

In [52]:
# 과천시 지역코드
df_gugun[df_gugun['gugun_nm'] == '과천시'] 

,gugun_nm,gugun_cd


In [53]:
# API를 통해 과천시 지점만 가져옴
data_과천 = {
    'ins_lat': '33.4996213', # 제주 시청의 좌표
    'ins_lng': '126.5311884',
    'p_sido_cd':'08', # 01=서울시, 08=경기 ... 16=제주
    'p_gugun_cd':'0803',  # 세부지역 (지정하지 않으면 시/도 전체)
    'in_biz_cd':'',
    'set_date':'',
    'iend':'1000',
}
    
url = 'https://www.istarbucks.co.kr/store/getStore.do'
r = requests.post(url, data=data_과천)
df_과천 = json_normalize(json.loads(r.text), 'list')
df_과천 = df_과천[['s_name', 'lat', 'lot', 'sido_name', 'gugun_name', 'doro_address', 'tel']]
df_과천.head(10)

,s_name,lat,lot,sido_name,gugun_name,doro_address,tel
0,과천이마트,37.42629945,126.991852,경기,과천시,경기도 과천시 별양상가3로 11 이마트 1층(별양동),02-758-8033
1,과천,37.428183,126.991074,경기,과천시,"경기도 과천시 중앙로 129, 고려빌딩 1층 (중앙동)",02-758-8369


In [54]:
# 경기 전체 스벅에서 과천시 지점만 출력
data_경기 = {
    'ins_lat': '33.4996213', # 제주 시청의 좌표
    'ins_lng': '126.5311884',
    'p_sido_cd':'08', # 01=서울시, 08=경기 ... 16=제주
    'p_gugun_cd':'',  # 세부지역 (지정하지 않으면 시/도 전체)
    'in_biz_cd':'',
    'set_date':'',
    'iend':'1000',
}
    
url = 'https://www.istarbucks.co.kr/store/getStore.do'
r = requests.post(url, data=data_경기)
df_경기 = json_normalize(json.loads(r.text), 'list')

df_경기 = df_경기[['s_name', 'lat', 'lot', 'sido_name', 'gugun_name', 'doro_address', 'tel']]
df_경기[df_경기['gugun_name'] == '과천시'] 

,s_name,lat,lot,sido_name,gugun_name,doro_address,tel
125,과천이마트,37.42629945,126.991852,경기,과천시,경기도 과천시 별양상가3로 11 이마트 1층(별양동),02-758-8033
127,과천,37.428183,126.991074,경기,과천시,"경기도 과천시 중앙로 129, 고려빌딩 1층 (중앙동)",02-758-8369


이로써 구군 지역 코드로 스타벅스 API를 사용하는 법을 확인함

### 스타벅스 API 필터
- 드라이브 스루, 리저브, 주차, 지하철 인접으로 필요해 보이는 필터들의 API param을 확인함

In [55]:
# http://www.istarbucks.co.kr/store/store_map.do 에서 개발자 도구로 네트워크 보면서 찾음
# 필터는 일단 필요해 보이는 것만 

filter = {
    'T01':0, # 드라이브 스루
    'T03':1, # 리저브
    'T06':0,
    'T09':0, # 주차
    'T10':0,
    'T12':0,
    'T22':0,
    'P80':0  # 지하철 인접
}

In [56]:
url = 'https://www.istarbucks.co.kr/store/getStore.do'
r = requests.post(url, data={**data, **filter})
df = json_normalize(json.loads(r.text), 'list')
df[['s_name', 'lat', 'lot', 'sido_name', 'gugun_name', 'doro_address', 'tel']]

,s_name,lat,lot,sido_name,gugun_name,doro_address,tel
0,수원이비스,37.2592235,127.0313138,경기,수원시 팔달구,경기도 수원시 팔달구 권광로 132 (인계동),031-224-5877
1,수원인계,37.265499,127.032902,경기,수원시 팔달구,경기도 수원시 팔달구 권광로 199 (인계동),031-226-8544
2,시흥P.아울렛 3F,37.379754,126.736396,경기,시흥시,경기도 시흥시 서해안로 699 (정왕동),031-8072-3672
3,정자,37.365045,127.106119,경기,성남시 분당구,"경기도 성남시 분당구 정자일로 166, SPG센터 (정자동)",031-8022-8559
4,평촌역사거리,37.395786,126.964500,경기,안양시 만안구,경기도 안양시 동안구 시민대로 311 금강스마트 빌딩 1층(관양동1746),031-426-8918
5,분당구청,37.38405,127.120257,경기,성남시 분당구,경기도 성남시 분당구 분당로 43 (서현동),031-708-6758
6,판교역,37.395911,127.113233,경기,성남시 분당구,"경기도 성남시 분당구 대왕판교로606번길 58, 판교푸르지오월드마크 1층 (삼평동)",031-8016-5666
7,야탑글라스,37.410845,127.129806,경기,성남시 분당구,경기도 성남시 분당구 성남대로916번길 11 (야탑동),031-8016-8416
8,부천신중동역,37.5037646,126.77414390000001,경기,부천시 오정구,경기도 부천시 길주로 281 (중동) 동화빌딩,032-322-4921
9,스타필드하남2FR,37.544033,127.225424,경기,하남시,경기도 하남시 미사대로 750 (신장동),031-8072-8339
